In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
response = requests.get("https://www.cookpolitical.com/ratings/house-race-ratings")
doc = BeautifulSoup(response.text, 'html.parser')

In [3]:
ratings = doc.find_all(class_='solid-seats-modal-in')

In [4]:
rating_list = []
for rating in ratings:
    for candidate in rating.find_all(class_='popup-table-data-row')[1:]:
        rating_dic = {}
        rating_dic['Rating'] = rating.find(class_='solid-seats-modal-in-title').text
        rating_dic['District'] = candidate.find_all(class_='popup-table-data-cell')[0].text
        rating_dic['Representative'] = candidate.find_all(class_='popup-table-data-cell')[1].text
        rating_list.append(rating_dic)

In [5]:
df = pd.DataFrame(rating_list)
df.District = df.District.str.extract('([A-Z]{2}-[\d]{2})')
df['STATEFP'] = df.District.str.extract('([A-Z]{2})')
df['CD115FP'] = df.District.str.extract('([\d]{2})')
df.head()

,District,Rating,Representative,STATEFP,CD115FP
0,AL-07,Solid Democratic,Terri Sewell,AL,07
1,AZ-03,Solid Democratic,Raúl Grijalva,AZ,03
2,AZ-07,Solid Democratic,Ruben Gallego,AZ,07
3,AZ-09,Solid Democratic,Open,AZ,09
4,CA-02,Solid Democratic,Jared Huffman,CA,02


In [6]:
df.Rating.value_counts()

Solid Democratic      181
Solid Republican      153
Toss-Up Republican     27
Lean Republican        26
Likely Republican      25
Likely Democratic      12
Lean Democratic         9
Toss-Up Democratic      2
Name: Rating, dtype: int64

In [7]:
state = pd.read_excel('state-geocodes-v2016.xls',skiprows=5,dtype=str)
state = state.iloc[:,2:].sort_values(by='Name')
state.columns = ['FIPS','State']
state = state[state.FIPS != '00'].reset_index(drop=True)
state.head()

,FIPS,State
0,01,Alabama
1,02,Alaska
2,04,Arizona
3,05,Arkansas
4,06,California


In [8]:
abb = pd.read_csv('states.csv')
abb.head()

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [9]:
df_merge = pd.merge(state,abb,on='State')
df_merge.head()

,FIPS,State,Abbreviation
0,01,Alabama,AL
1,02,Alaska,AK
2,04,Arizona,AZ
3,05,Arkansas,AR
4,06,California,CA


In [10]:
for i in range(0,len(df)):
    for j in range(0,len(df_merge)):
        if df['STATEFP'][i] == df_merge['Abbreviation'][j]:
            df['STATEFP'][i] = df_merge['FIPS'][j]

In [11]:
df['AFFGEOID'] = '5001500US' + df['STATEFP'] + df['CD115FP']
df.head()

,District,Rating,Representative,STATEFP,CD115FP,AFFGEOID
0,AL-07,Solid Democratic,Terri Sewell,01,07,5001500US0107
1,AZ-03,Solid Democratic,Raúl Grijalva,04,03,5001500US0403
2,AZ-07,Solid Democratic,Ruben Gallego,04,07,5001500US0407
3,AZ-09,Solid Democratic,Open,04,09,5001500US0409
4,CA-02,Solid Democratic,Jared Huffman,06,02,5001500US0602


In [12]:
df['Color'] = df.Rating.replace({'Solid Republican':1,'Likely Republican':2,'Lean Republican':3,'Toss-Up Republican':4,'Toss-Up Democratic':5,'Lean Democratic':6,'Likely Democratic':7,'Solid Democratic':8})
df.head()

,District,Rating,Representative,STATEFP,CD115FP,AFFGEOID,Color
0,AL-07,Solid Democratic,Terri Sewell,01,07,5001500US0107,8
1,AZ-03,Solid Democratic,Raúl Grijalva,04,03,5001500US0403,8
2,AZ-07,Solid Democratic,Ruben Gallego,04,07,5001500US0407,8
3,AZ-09,Solid Democratic,Open,04,09,5001500US0409,8
4,CA-02,Solid Democratic,Jared Huffman,06,02,5001500US0602,8


In [13]:
df.to_csv('Rating.csv',index=False)